In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()

# Obtener el contenido de una variable
path = os.getenv('CHROME_PATH') # Para evitar compartir nuestro path

In [5]:
# Establecemos el driver
chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")

# Ruta al chromedriver
chromedriver_path = fr'{path}'

# Inicializa el navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [6]:
# Entramos dentro de la pagina a scrapear, maximizamos la pantalla y aceptamos las cookies

driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers') # Las reseñas las vamos a obtener de los bestsellers
driver.maximize_window()

accept_cokies = driver.find_element(By.ID ,'sp-cc-accept')
accept_cokies.click()
driver.implicitly_wait(10) 

In [4]:
# Dentro de la pagina buscamos todas las urls "ver mas" para poder scrapear la mayor cantidad de productos en la menor cantidad de enlaces posibles
 
urls_econtradas = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
urls_ver_mas = []
for url in urls_econtradas:
    if url.text == 'Ver más':
        urls_ver_mas.append(url)

In [5]:
def obtener_reseñas(soup): # recibimos una lista de reseñas
    clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
    reseñas_en_div = soup.find_all('div', class_=clase_producto)

    reseñas = [] # Una lista con las reseñas de cada producto
    for reseña in reseñas_en_div:
        if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
            # y otras nos daban nones, por lo que podemos filtrar un poco
            reseñas.append(reseña.find('span').text)
            driver.implicitly_wait(10)

    return reseñas

In [7]:
def dataframe_por_producto(categoria): # recibiremos un pandas con todas las reseñas de un producto, el nombre del producto y su categoría
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        nombre_producto = soup.find('span', id='productTitle').text

        reseñas = obtener_reseñas(soup) # obtenemos la lista de todas las reseñas de un producto

        # Creamos el dataframe
        producto = pd.DataFrame()
        producto['reseñas_humanas'] = reseñas
        producto['nombre_producto'] = nombre_producto
        producto['categoria_producto'] = categoria
        driver.implicitly_wait(10)

        return producto

    except Exception as e:
        error = f"Error al extraer información: {e}"
        return error

In [8]:
# Basicamente devolvera un dataframe que es la unión de los dataframes de todos los productos dentro de una categoría
def dataframe_por_categorias():
    try:   
        urls_todos_productos = driver.find_elements(By.CLASS_NAME, 'aok-block')
        driver.implicitly_wait(10)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
        categoria = categoria_en_div.find('h1').text

        lista_df = [] # Una lista con los dataframes de cada producto
        urls = [elemento.get_attribute('href') for elemento in urls_todos_productos] # Forma segura de tratar las urls
        driver.implicitly_wait(10)

        for url_producto in urls:
            driver.get(url_producto)
            driver.implicitly_wait(10)
            
            producto = dataframe_por_producto(categoria)
            lista_df.append(producto)
            
            driver.back()
            driver.implicitly_wait(10)

        combined_df = pd.concat(lista_df, ignore_index=True) # La unión de todos los dataframes
        return combined_df
    
    except Exception as e:
        print(f"Error al extraer información de {url_producto}: {e}") 

In [9]:
lista_combined_df = [] # Es una lista con los dataframes de cada categoria
for url_ver_mas in urls_ver_mas:
    
    url_ver_mas.click() # accedemos a cada url
    driver.implicitly_wait(10)

    combined_df = dataframe_por_categorias() # obtenemos un dataframe con todas las reseñas de cada categoría
    lista_combined_df.append(combined_df)

    driver.back()
    driver.implicitly_wait(10)

driver.quit()

In [10]:
df_definitivo = pd.concat(lista_combined_df, ignore_index=True) # El dataframe final que sera la unión de todos los dataframes de las categorías

In [11]:
df_definitivo

,reseñas_humanas,nombre_producto,categoria_producto
0,Necesitaba un nuevo movil y me decidí por este...,"Apple iPhone 12, 128GB, Negro - (Reaco...",Los más vendidos en Amazon Renewed
1,Quisiera expresar mi profunda insatisfacción c...,"Apple iPhone 12, 128GB, Negro - (Reaco...",Los más vendidos en Amazon Renewed
2,Está en perfectas condiciones la batería al 10...,"Apple iPhone 12, 128GB, Negro - (Reaco...",Los más vendidos en Amazon Renewed
3,Me llegó ayer el iPhone en perfectas condicion...,"Apple iPhone 12, 128GB, Negro - (Reaco...",Los más vendidos en Amazon Renewed
4,El iPhone 12 es un auténtico logro de la ingen...,"Apple iPhone 12, 128GB, Negro - (Reaco...",Los más vendidos en Amazon Renewed
...,...,...,...
2243,"Produkt świetny, do pełni szczęścia brakuje ap...","Luz de Pared de Interior, Lámpara de P...",Los más vendidos en Iluminación
2244,bel effet,"Luz de Pared de Interior, Lámpara de P...",Los más vendidos en Iluminación
2245,As mentioned in the title... Had to buy new li...,"Luz de Pared de Interior, Lámpara de P...",Los más vendidos en Iluminación
2246,"Ne ho comprate due,una per me e una per i miei...","Luz de Pared de Interior, Lámpara de P...",Los más vendidos en Iluminación


In [13]:
df_definitivo.to_csv('data/reseñas_amazon.csv', index=False)